# Análise do Titanic - Parte 5

## Importando as bases de dados de treino e teste

In [167]:
import pandas as pd

treino = pd.read_csv('Parte 2/base_parte2.csv')
teste = pd.read_csv('Parte 2/base_teste_parte2.csv')

In [168]:
treino.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1


In [169]:
treino = treino.drop('Sex', axis=1)
teste = teste.drop('Sex', axis=1)

In [170]:
from sklearn.preprocessing import RobustScaler

transformer = RobustScaler().fit(treino[['Age', 'Fare']])
treino[['Age', 'Fare']] = transformer.transform(treino[['Age', 'Fare']])

transformer_teste = RobustScaler().fit(teste[['Age', 'Fare']])
teste[['Age', 'Fare']] = transformer_teste.transform(teste[['Age', 'Fare']])

In [171]:
treino.head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,-0.592240,1,0,-0.312011,S,1,0,0,1
1,2,1,1,0.638529,1,0,2.461242,C,0,1,0,0
2,3,1,3,-0.284548,0,0,-0.282777,S,0,0,0,1


In [172]:
def sozinho(a, b):
    if (a == 0 and b == 0):
        return 1
    else:
        return 0

In [173]:
treino['Sozinho'] = treino.apply(lambda x: sozinho(x.SibSp, x.Parch), axis=1)

teste['Sozinho'] = teste.apply(lambda x: sozinho(x.SibSp, x.Parch), axis=1)

In [174]:
treino['Familiares'] = treino.SibSp + treino.Parch

teste['Familiares'] = teste.SibSp + teste.Parch

In [175]:
treino.head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S,Sozinho,Familiares
0,1,0,3,-0.592240,1,0,-0.312011,S,1,0,0,1,0,1
1,2,1,1,0.638529,1,0,2.461242,C,0,1,0,0,0,1
2,3,1,3,-0.284548,0,0,-0.282777,S,0,0,0,1,1,0


In [176]:
treino2 = pd.read_csv('DataBase/train.csv')
teste2 = pd.read_csv('DataBase/test.csv')

treino2.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [177]:
treino2['Embarked'] = treino2['Embarked'].fillna('S')

In [178]:
from sklearn.preprocessing import OrdinalEncoder

categorias = ['S', 'C', 'Q']

enc = OrdinalEncoder(categories=[categorias], dtype='int32')

enc = enc.fit(treino2[['Embarked']])

In [179]:
# Voltando para a base de treino original
treino['Embarked'] = enc.transform(treino2[['Embarked']])

In [180]:
enc = enc.fit(teste2[['Embarked']])

In [181]:
# Voltando para a base de teste original
teste['Embarked'] = enc.transform(teste2[['Embarked']])

In [182]:
treino.head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S,Sozinho,Familiares
0,1,0,3,-0.592240,1,0,-0.312011,0,1,0,0,1,0,1
1,2,1,1,0.638529,1,0,2.461242,1,0,1,0,0,0,1
2,3,1,3,-0.284548,0,0,-0.282777,0,0,0,0,1,1,0


In [183]:
treino = treino.drop(['Embarked_C', 'Embarked_S', 'Embarked_Q'], axis=1)

teste = teste.drop(['Embarked_C', 'Embarked_S', 'Embarked_Q'], axis=1)

In [184]:
treino.head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Sozinho,Familiares
0,1,0,3,-0.592240,1,0,-0.312011,0,1,0,1
1,2,1,1,0.638529,1,0,2.461242,1,0,0,1
2,3,1,3,-0.284548,0,0,-0.282777,0,0,1,0


### Nesta parte iremos testar outro modelos para realizar a previsão

* Nas partes anteriores, nós estávamos focando em: Árvore de Decisão, KNN e Regressão Logística. Dos três, Regressão Logística apresentou uma maior acurácia, logo iremos manter.

* Outro modelos disponíveis são:
    - Random Forest

    - MLPClassifier (Redes Neurais)

* Para tentarmos melhorar o modelo em comparação com o utilizado na Parte 4, vamos atribuir além do **train_test_split** o **grid_search** para estimar os melhores parâmetros

In [185]:
from sklearn.model_selection import train_test_split

In [186]:
X = treino.drop(['PassengerId', 'Survived'], axis=1)
y = treino.Survived

In [187]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

* Regressão Logística

In [188]:
from sklearn.linear_model import LogisticRegression

clf_rl = LogisticRegression(random_state=42)

clf_rl = clf_rl.fit(X_train, y_train)

In [189]:
# Definindo os parâmtros do modelo de Regressão Logística
parametros_rl = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'max_iter': [100, 1000, 5000, 10000]
}

* Random Forest

In [190]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(random_state=42)

clf_rf = clf_rf.fit(X_train, y_train)

In [191]:
# Definindo os parâmtros do modelo de Random Forest
parametros_rf = {
    'n_estimators': [100, 200, 500, 1000],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [2, 4, 6, 8, None],
    'max_features': ['sqrt', 'log2', None]
}

* MLPClassifier (Redes Neurais)

In [192]:
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier(random_state=42, max_iter=5000)

clf_mlp = clf_mlp.fit(X_train, y_train)

In [193]:
# Definindo os parâmtros do modelo de MLPClassifier
parametros_mlp = {
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [10.0**(-1), 10.0**(-5), 10.**(-7), 10.0**(-10)],
    'max_iter': [200, 500, 1000, 5000]
}

* Fazendo a busca pelos melhores parâmetros com **grid_search**

In [194]:
# Ignorar os avisos
import warnings
warnings.filterwarnings('ignore')

In [195]:
# Importanto o datetime para cronometrar a demora dos modelos
from datetime import datetime
def hora_atual():
    agora = datetime.now()
    print(str(agora.hour)+':'+str(agora.minute)+':'+str(agora.second))


In [196]:
# Importanto KFold e GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [197]:
# Para a Regressão Logística
hora_atual()
kfold_rl = KFold(random_state=42, shuffle=True, n_splits=8) # Separar as bases

grid_search_rl = GridSearchCV(clf_rl, parametros_rl, scoring='accuracy', cv=kfold_rl) # Verificar os melhores parâmetros
grid_search_rl = grid_search_rl.fit(X_train, y_train)

hora_atual()

14:3:41
14:3:48


In [198]:
# Para a Random Forest
hora_atual()
kfold_rf = KFold(random_state=42, shuffle=True, n_splits=8) # Separar as bases

grid_search_rf = GridSearchCV(clf_rf, parametros_rf, scoring='accuracy', cv=kfold_rf) # Verificar os melhores parâmetros
grid_search_rf = grid_search_rf.fit(X_train, y_train)

hora_atual()

14:3:48
14:16:50


In [199]:
# Para a MLPClassifier
hora_atual()
kfold_mlp = KFold(random_state=42, shuffle=True, n_splits=8) # Separar as bases

grid_search_mlp = GridSearchCV(clf_mlp, parametros_mlp, scoring='accuracy', cv=kfold_mlp) # Verificar os melhores parâmetros
grid_search_mlp = grid_search_mlp.fit(X_train, y_train)

hora_atual()

14:16:51
14:24:31


### Verificando os melhores parâmetros para cada modelo

In [222]:
# Melhor parâmetro da Regressão Logística
grid_search_rl.best_params_

{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}

In [223]:
# Melhor parâmetro da Random Forest
grid_search_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [224]:
# Melhor parâmetro da MLPClassifier
grid_search_mlp.best_params_

{'alpha': 0.1, 'max_iter': 200, 'solver': 'adam'}

### Verificando os scores para cada modelo

In [225]:
# Melhor score da Regressão Logística
grid_search_rl.best_score_

0.8089887640449438

In [226]:
# Melhor score da Random Forest
grid_search_rf.best_score_

0.8314606741573034

In [227]:
# Melhor score da MLPClassifier
grid_search_mlp.best_score_

0.8174157303370786

### Previsão nos dados de validação com os melhores parâmetros para cada modelo

In [228]:
# Regressão Logística
clf_best_rl = grid_search_rl.best_estimator_

y_pred_rl = clf_best_rl.predict(X_val)

In [229]:
# Random Forest
clf_best_rf = grid_search_rf.best_estimator_

y_pred_rf = clf_best_rf.predict(X_val)

In [230]:
# MLPClassifier
clf_best_mlp = grid_search_mlp.best_estimator_

y_pred_mlp = clf_best_mlp.predict(X_val)

### Avaliação dos modelos

* Acurácia

In [231]:
from sklearn.metrics import accuracy_score

In [232]:
# Regressão Logística
as_rl = accuracy_score(y_val, y_pred_rl)
as_rl

0.8044692737430168

In [233]:
# Random Forest
as_rf = accuracy_score(y_val, y_pred_rf)
as_rf

0.8100558659217877

In [234]:
# Redes Neurais
as_mlp = accuracy_score(y_val, y_pred_mlp)
as_mlp

0.8100558659217877

* Matriz de Confusão

In [235]:
from sklearn.metrics import confusion_matrix

In [236]:
# Regressão Logística
cm_rl = confusion_matrix(y_val, y_pred_rl)
cm_rl

array([[91, 14],
       [21, 53]], dtype=int64)

In [237]:
# Random Forest
cm_rf = confusion_matrix(y_val, y_pred_rf)
cm_rf

array([[93, 12],
       [22, 52]], dtype=int64)

In [238]:
# Redes Neurais
cm_mlp = confusion_matrix(y_val, y_pred_mlp)
cm_mlp

array([[92, 13],
       [21, 53]], dtype=int64)

* Nota-se que, os modelos de Redes Neurais utilizando MLPClassifier e Random Forest possuem a mesma taxa de acerto.

### Fazendo a previsão para os dados de teste

In [242]:
X_teste = teste.drop(['PassengerId', 'Survived'], axis=1)

In [246]:
y_pred = clf_best_rf.predict(X_teste)

In [247]:
teste['Survived'] = y_pred

In [248]:
base_envio = teste[['PassengerId', 'Survived']]

base_envio.to_csv('titanic_previsao5_rf.csv', index=False)

* Como ambos modelos apresentaram mesma acurácia, obteve-se uma mesma pontuação no Kaggle de 0.77990